# This code reads the grid which has risk level and suitable_f and includes all other datasets to the grid and then creates the exact grid. 
## THe data I am using here: 
1. Infrastrucutre / Detailed Infrastrucutre
2. Erosion
3. InSAR
4. Drained Lakes
5. Ice wedges
6. Contaminated sites


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


/home/emine2/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
grid_50 = gpd.read_file('/home/emine2/R121/Wainwright/grids/wainwright_grid_50m.shp')
grid_50.head()

,risk_level,suitable_f,geometry
0,0.0,True,"POLYGON ((-243897.77 2303949.894, -243897.77 2..."
1,0.0,True,"POLYGON ((-243897.77 2303999.894, -243897.77 2..."
2,0.0,True,"POLYGON ((-243897.77 2304049.894, -243897.77 2..."
3,0.0,True,"POLYGON ((-243897.77 2304099.894, -243897.77 2..."
4,0.0,True,"POLYGON ((-243897.77 2304149.894, -243897.77 2..."


## 1. Infrastructure

In [3]:
infrastructure = gpd.read_file('/home/emine2/R121/Wainwright/infrastructure/detailed_infrastructure/w_infrastructure/w_detailed_infrastructure.shp')
infrastructure.head()

,OBJECTID,Community,Infr_Type,Source,SHAPE_Leng,Infr_Group,geometry
0,21892.0,Wainwright,Centerline,DCRA,141.673006,Transportation,"LINESTRING (-227838.135 2304987.372, -227739.7..."
1,21893.0,Wainwright,Centerline,DCRA,163.122196,Transportation,"LINESTRING (-227643.014 2304942.695, -227756.2..."
2,21894.0,Wainwright,Centerline,DCRA,499.010263,Transportation,"LINESTRING (-228047.42 2304801.45, -227674.355..."
3,21895.0,Wainwright,Centerline,DCRA,23.698657,Transportation,"LINESTRING (-227613.418 2305159.294, -227629.8..."
4,21896.0,Wainwright,Centerline,DCRA,112.433303,Transportation,"LINESTRING (-227550.97 2305211.291, -227472.93..."


In [4]:
# Initialize new columns
grid_50['infra_exist'] = 0
grid_50['infra_type'] = None

# Perform spatial join to find intersections
intersections = gpd.sjoin(grid_50, infrastructure, predicate='intersects')

# Get unique grid indices that intersect
intersecting_grids = intersections.index.unique()

# Set infra_exist to 1 for intersecting grids
grid_50.loc[intersecting_grids, 'infra_exist'] = 1

# Group by grid index and get unique infra types as comma-separated string
types_per_grid = intersections.groupby(level=0)['Infr_Type'].unique().apply(lambda x: ', '.join(sorted(x)))

# Set infra_type for intersecting grids
grid_50.loc[types_per_grid.index, 'infra_type'] = types_per_grid

# Optionally, view the updated head
grid_50.head()

/tmp/ipykernel_1388947/3837121425.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3338
Right CRS: EPSG:6393

  intersections = gpd.sjoin(grid_50, infrastructure, predicate='intersects')


,risk_level,suitable_f,geometry,infra_exist,infra_type
0,0.0,True,"POLYGON ((-243897.77 2303949.894, -243897.77 2...",0,None
1,0.0,True,"POLYGON ((-243897.77 2303999.894, -243897.77 2...",0,None
2,0.0,True,"POLYGON ((-243897.77 2304049.894, -243897.77 2...",0,None
3,0.0,True,"POLYGON ((-243897.77 2304099.894, -243897.77 2...",0,None
4,0.0,True,"POLYGON ((-243897.77 2304149.894, -243897.77 2...",0,None


## 2. Erosion Forecast

In [5]:
erosion_forecast = gpd.read_file('/home/emine2/R121/Wainwright/erosion/W_erosion_forecast/W_erosion_forecast.shp')
erosion_forecast.head()

,OBJECTID,Community,YearStart,YearEnd,SHAPE_Leng,SHAPE_Area,geometry
0,118.0,Wainwright,2019,2039,17303.977273,45640.651803,"MULTIPOLYGON (((-230112.195 2303099.186, -2301..."
1,119.0,Wainwright,2039,2059,17302.613705,44954.637131,"MULTIPOLYGON (((-230175.342 2303050.278, -2301..."
2,120.0,Wainwright,2059,2079,17337.070417,44979.079859,"MULTIPOLYGON (((-230174.878 2303049.777, -2301..."


In [ ]:
# Initialize new columns
grid_50['erosion_exist'] = 0

# Perform spatial join to find intersections
intersections = gpd.sjoin(grid_50, erosion_forecast, predicate='intersects')

# Get unique grid indices that intersect
intersecting_grids = intersections.index.unique()

# Set erosion_exist to 1 for intersecting grids
grid_50.loc[intersecting_grids, 'erosion_exist'] = 1

# Group by grid index and get unique YearStart values as comma-separated string
years_per_grid = intersections.groupby(level=0)['YearStart'].unique().apply(lambda x: ', '.join(map(str, sorted(x))))

# Set erosion_year for intersecting grids
grid_50.loc[years_per_grid.index, 'erosion_year'] = years_per_grid

# Optionally, view the updated head
grid_50.head()

/tmp/ipykernel_1388947/2512283771.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3338
Right CRS: EPSG:6393

  intersections = gpd.sjoin(grid_50, erosion_forecast, predicate='intersects')


   risk_level  suitable_f                                           geometry  \
0         0.0        True  POLYGON ((-243897.77 2303949.894, -243897.77 2...   
1         0.0        True  POLYGON ((-243897.77 2303999.894, -243897.77 2...   
2         0.0        True  POLYGON ((-243897.77 2304049.894, -243897.77 2...   
3         0.0        True  POLYGON ((-243897.77 2304099.894, -243897.77 2...   
4         0.0        True  POLYGON ((-243897.77 2304149.894, -243897.77 2...   

   infra_exist infra_type  erosion_exist erosion_year  
0            0       None              0          NaN  
1            0       None              0          NaN  
2            0       None              0          NaN  
3            0       None              0          NaN  
4            0       None              0          NaN  
